# Packages

In [1]:
%pip install statsbombpy


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
import ast

import pandas as pd
import geopandas as gpd
import numpy as np
from statsbombpy import sb

In [18]:
PROJECT_ROOT = Path.cwd().resolve().parent
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
RAW_DIR = DATA_DIR / "raw"

# Loading data

Getting the competition ids

In [17]:
# loading the liga games from the 2015/16 season
competitions = sb.competitions()[(sb.competitions()["season_name"] == "2015/2016") & (sb.competitions()["competition_name"] != "Champions League")]["competition_id"]
season_id = 27 # for 2015/16 season
sb.competitions()[(sb.competitions()["competition_id"].isin(competitions)) & (sb.competitions()["season_id"] == season_id)]["competition_name"]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


1      1. Bundesliga
43           La Liga
60           Ligue 1
64    Premier League
66           Serie A
Name: competition_name, dtype: object

In [24]:
match_ids = {}
for competition_id in competitions:
    match_ids[competition_id] = list(sb.matches(competition_id=competition_id, season_id=season_id)["match_id"])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [30]:
match_ids

{9: [3890561,
  3890505,
  3890511,
  3890515,
  3890411,
  3890397,
  3890401,
  3890396,
  3890384,
  3890385,
  3890282,
  3890287,
  3890269,
  3890268,
  3890402,
  3890564,
  3890563,
  3890562,
  3890560,
  3890559,
  3890558,
  3890557,
  3890556,
  3890555,
  3890554,
  3890553,
  3890552,
  3890551,
  3890550,
  3890549,
  3890548,
  3890547,
  3890546,
  3890545,
  3890544,
  3890543,
  3890542,
  3890541,
  3890540,
  3890539,
  3890538,
  3890537,
  3890536,
  3890535,
  3890534,
  3890533,
  3890532,
  3890531,
  3890530,
  3890529,
  3890528,
  3890527,
  3890526,
  3890525,
  3890524,
  3890523,
  3890522,
  3890521,
  3890520,
  3890519,
  3890518,
  3890517,
  3890516,
  3890514,
  3890513,
  3890512,
  3890510,
  3890509,
  3890508,
  3890507,
  3890506,
  3890504,
  3890503,
  3890502,
  3890501,
  3890500,
  3890499,
  3890498,
  3890497,
  3890496,
  3890495,
  3890494,
  3890493,
  3890492,
  3890491,
  3890490,
  3890489,
  3890488,
  3890487,
  3890486,
  38904

In [51]:
events = pd.DataFrame()

for key, match_id_list in match_ids.items():
    for match_id in match_id_list:
        shot_events = sb.events(match_id, filters={"type.name": "Shot"})
        events = pd.concat([events, shot_events], axis=0, ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Pytho

In [52]:
events

,duration,id,index,location,match_id,minute,period,play_pattern,player,player_id,...,type,under_pressure,out,shot_deflected,shot_open_goal,shot_redirect,shot_saved_off_target,shot_saved_to_post,off_camera,shot_follows_dribble
0,0.597523,ba46e9d6-e828-4599-952c-39c1f7d22659,263,"[108.1, 31.2]",3890561,4,1,Regular Play,Tarik Elyounoussi,26832,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.636048,85d67225-30fb-47c8-b478-cf568941a164,353,"[110.9, 42.6]",3890561,6,1,From Throw In,Klaas-Jan Huntelaar,8060,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.125672,adac17d3-5e67-4e8c-b482-4bae2f36e06e,410,"[117.9, 29.1]",3890561,8,1,From Throw In,Kevin Volland,8215,...,Shot,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.303079,abffd193-62bc-4c8d-8636-1e3f0f0ebbe5,624,"[101.8, 27.6]",3890561,13,1,From Counter,Jean-Eric Maxim Choupo-Moting,3499,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.666134,d9cea903-f92a-40e1-a393-1a849d83f157,749,"[109.3, 26.5]",3890561,17,1,Regular Play,Klaas-Jan Huntelaar,8060,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45714,0.074412,2d60277a-da24-4e8a-aa97-42f2cb0830fa,3758,"[98.6, 40.4]",3878540,89,2,Regular Play,Danilo Cataldi,7107,...,Shot,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45715,0.483935,7d4e50ce-7ecd-44a1-b242-d1001d05417a,3785,"[103.0, 51.3]",3878540,90,2,From Free Kick,Franco Brienza,45591,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45716,0.995026,fa4f5417-a44e-4d14-b4f5-05925d778a07,3789,"[103.8, 51.4]",3878540,90,2,From Free Kick,Franco Brienza,45591,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45717,0.765232,7268aefc-277a-40bc-9995-5830039d6a5e,3792,"[110.0, 39.4]",3878540,90,2,From Free Kick,Matteo Brighi,18027,...,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
grouped_events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2015/2016",
    split=True
)
grouped_events["shots"]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Library/Frameworks/Pytho

KeyboardInterrupt: 